# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [39]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
# import cartopy.crs as ccrs
# import geoviews as gv # noqa
# import pyproj
# Import API key
from api_keys import geoapify_key

In [40]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,codrington,-38.2667,141.9667,12.27,84,100,5.28,AU,1681585423
1,1,palmer,42.1584,-72.3287,25.92,48,75,3.13,US,1681585191
2,2,bentiu,9.2333,29.8333,31.41,41,27,2.72,SS,1681585424
3,3,tuktoyaktuk,69.4541,-133.0374,-17.00,85,75,4.12,CA,1681585424
4,4,punta arenas,-53.1500,-70.9167,7.06,81,75,4.12,CL,1681585372
...,...,...,...,...,...,...,...,...,...,...
538,538,hvide sande,56.0045,8.1294,11.72,88,95,3.99,DK,1681585547
539,539,punta alta,-38.8759,-62.0736,21.71,40,51,1.36,AR,1681585547
540,540,jiazi,22.8779,116.0670,22.44,87,0,3.07,CN,1681585547
541,541,srednekolymsk,67.4500,153.6833,-19.13,100,53,2.33,RU,1681585547


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [41]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
    #tiles = "OSM"
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [42]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_temp_df = city_data_df.loc[(city_data_df['Max Temp'] >23) & (city_data_df['Max Temp'] < 25) & (city_data_df["Humidity"]==94)]
ideal_temp_df
# Drop any rows with null values
ideal_temp_df.dropna()




,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
105,105,kendari,-3.9450,122.4989,24.28,94,91,1.08,ID,1681585445
297,297,westport,41.1415,-73.3579,23.69,94,100,4.63,US,1681585207


### Step 3: Create a new DataFrame called `hotel_df`.

In [43]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_temp_df.iloc[:, [1,2,3,5,8]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""
# Display sample data
hotel_df.head()

,City,Lat,Lng,Humidity,Country,Hotel Name
105,kendari,-3.9450,122.4989,94,ID,
297,westport,41.1415,-73.3579,94,US,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [44]:
# Set parameters to search for a hotel

radius = 1000
limit = 20
distance = 10000
params = {
    "limit":limit,
    "distance":distance,
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df.loc[index, "Lat"]
    lng = hotel_df.loc[index, "Lng"]
  
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kendari - nearest hotel: No hotel found
westport - nearest hotel: No hotel found


,City,Lat,Lng,Humidity,Country,Hotel Name
105,kendari,-3.9450,122.4989,94,ID,No hotel found
297,westport,41.1415,-73.3579,94,US,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [45]:
%%capture --no-display

# Configure the map plot
map_hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    hotel_name = "Hotel Name",
    country = "Country",
    hover_cols=["Hotel Name", "Country"]
    #tiles = "OSM"
)

# Display the map
map_hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)